In [4]:
%load_ext autoreload
%autoreload 2
import os
import re
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write
#from pipefunc.pipefunc import do_command, send_command, get_response, get_response_json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
p = Path(os.getcwd())
p_raw = p.parents[0].joinpath('raw')
p_mono = p.parents[0].joinpath('mono')
p_mono.mkdir(exist_ok=True)
wavfilelist = sorted(p_raw.glob('*.wav'))
print(wavfilelist)
op_monofilelist = [p_mono.joinpath(x.name) for x in wavfilelist]

[WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Aileen.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Mike.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_Robin.wav'), WindowsPath('c:/Users/michaelshih/Documents/Podcast_tmp/EP019/raw/Aileen_WWL.wav')]


In [ ]:
if p_raw.joinpath('metadata.json').is_file():
    with open(p_raw.joinpath('metadata.json'), 'r') as f:
        metadata_dict = json.load(f)
else:    
    metadata_dict = {}
    for wavfile in wavfilelist:
        print(wavfile)
        fs, data = read(wavfile)
        print(fs)
        print(data.shape)
        metadata_dict[wavfile.name] = {'fs': fs, 
                                        'shape': data.shape,
                                        }
    with open(p_raw.joinpath('metadata.json'), 'w') as f:
        json.dump(metadata_dict, f)

print(metadata_dict)
audiolength = []
for wavfile, value in metadata_dict.items():
    audiolength.append(value['shape'][0])
print(audiolength)
maxlength = np.amax(audiolength)

for wavfile, op_monofile in zip(wavfilelist, op_monofilelist):
    if not op_monofile.is_file():
        fs, data = read(wavfile)
        data_tmp = np.pad(data[:, 0], (0, maxlength - data.shape[0]), 'constant')
        write(op_monofile, fs, data[:, 0])